In [1]:
import networkx as nx
from bs4 import BeautifulSoup
import re
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import requests
import os
import numpy as np
from operator import itemgetter

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
dataYears = ["18", "17", "16", "15", "14", "13"]
dataMonths = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
dataMonths2Dig = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
dataPath = Path("data")

In [4]:
#from txt to csv, eliminating unnecessary data, after this piece of code is executed, the data for months is situated in aptly named folders dec13, jan14 etc.

def dataPreprocessing():
    dataMonthsRev = list(reversed(dataMonths)) # reversing the list because pre august 2016 data had a different organisation within the file
    changeInDataset = True
    ratingLimit = 2000

    for year in dataYears[::-1]:
        for month in dataMonths:
            folderName = month + year
            if folderName == "sep16":
                changeInDataset = False

            fileName = "standard_" + folderName + "frl.txt"
            with open(dataPath/folderName/fileName, 'r') as myfile:
                with open(dataPath/folderName/(folderName + ".csv"), 'w') as writeFile:
                    people = []
                    lines = myfile.readlines()
                    for line in tqdm(lines):
                        id = line[:15].strip()
                        
                        if id == 'ID Number': # eliminate file headers
                            continue
                        
                        name = line[15:76].strip()
                        name = name.replace(',', ';', 10) # replace all commas in name with semicolons, because comma is a delimiter in csv file
                        
                        nat = line[76:80].strip()
                        gender = line[80]
                        
                        if changeInDataset:
                            rating_str = line[109:115].strip()
                        else:
                            rating_str = line[113:118].strip()
                        
                        if len(rating_str) > 0:
                            rating = int(rating_str)
                            
                        if changeInDataset:
                            games_played = line[115:118].strip()
                        else:
                            games_played = line[119:122].strip()
            
#                         if len(games_played_str) > 0:
#                             games_played = int(games_played_str)
                        
                        if changeInDataset:
                            birth = line[122:126].strip()
                        else:
                            birth = line[126:130].strip()
                            
#                         if len(birth_str) > 0:
#                             birth = int(birth_str)
                        
                        if changeInDataset:
                            activity = line[128:].strip()
                        else:
                            activity = line[132:].strip()

                        if 'i' not in activity and len(rating_str) > 0 and rating > ratingLimit: # appending every entry to a list for sorting later
                            people.append((id, name, nat, rating_str, gender, games_played, birth, activity))
                    
                    people.sort(key=lambda tup: int(tup[0])) # sort all people by their FIDE identification 
                    for entry in people:
                        writeFile.write(entry[0] + "," + entry[1] + "," + entry[2] + "," + entry[3] + "," + entry[4] + "," + entry[5] + "," + entry[6] + "," + entry[7] + '\n')

#ratings in the constructed txt files are for the end of the month and should be used for building the next month's graph
#dataPreprocessing()

In [67]:

#fetching matches of a particular player commited for the specified month (many of these mathes were, possibly, played several months before, howbeit, were only commited now)

#id = FIDE id, date = string in yyyy-mm-01 format, returns adjacency list in form ("surname, name", "FIDE rating")
def fetchMatches(id, date):
    targetLink = "https://ratings.fide.com/individual_calculations.phtml?idnumber="+str(id)+"&rating_period="+date
    #page = requests.get("https://ratings.fide.com/individual_calculations.phtml?idnumber=2020009&rating_period=2015-11-01")
    page = requests.get(targetLink)
    soup = BeautifulSoup(page.content, 'html.parser')
    adjacencyDict = {}

    #fetch tournaments table "by rope and stick"
    tournament_table_tbody = soup.find('tr',  {"bgcolor": "#CC9966"}).parent
    
    rows = tournament_table_tbody.findAll("tr")
    tournament_rating = 0
    tournament_date = ''
    for tr in rows:
        if tr.has_attr('bgcolor'):
            if tr['bgcolor']=="#CC9966":               
                #for each tournament get month when it started
                tournament_date = tr.findAll('td')[3].contents[0].strip()
                tournament_date = tournament_date[:-2]+"01"
            if tr['bgcolor']=="#e6e6e6":
                tournament_rating = tr.findAll("td")[1].contents[0].strip()
                
        
        tdList = tr.findAll("td", {"class": "list4"})
        for td in tdList:
            if td.img:
                img_src = td.img.get('src')
                #print(img_src)
                if img_src == "/imga/clr_bl.gif":
                    color = "b"
                else: 
                    color = "w"
                td.img.decompose()

                cnt = 1
                for child in td.children:
                #print("Child ", cnt, ": ", child)
                    if cnt == 1:
                        name = child.strip()
                    if cnt == 4:
                        if child.font: #if * exists in the rating
                            rescaling = 1
                        else:
                            rescaling = 0
                        rating = child.contents[0].strip()
                    if cnt == 6:
                        result = child.contents[0].strip()
                    if cnt == 8:
                        transfer = child.contents[0].strip()
                    if cnt == 11:
                        if tournament_date in adjacencyDict.keys():
                            adjacencyDict[tournament_date].append((name, rating, tournament_rating, rescaling, color, result, transfer))
                        else:
                            adjacencyDict[tournament_date] = [(name, rating, tournament_rating, rescaling, color, result, transfer)]
                    cnt = cnt + 1
            
    return adjacencyDict
    
print(fetchMatches(2020009, "2015-11-01"))

{'2015-09-01': [('Zaibi, Amir', '2408', '2808', 1, 'w', '1.00', '0.08'), ('Zaibi, Amir', '2408', '2808', 1, 'b', '1.00', '0.08'), ('Mamedov, Rauf', '2657', '2808', 0, 'b', '1.00', '0.30'), ('Mamedov, Rauf', '2657', '2808', 0, 'w', '0.50', '-0.20'), ('Kovalyov, Anton', '2616', '2808', 0, 'w', '1.00', '0.25'), ('Kovalyov, Anton', '2616', '2808', 0, 'b', '0.50', '-0.25'), ('Mamedyarov, Shakhriyar', '2736', '2808', 0, 'b', '0.00', '-0.60'), ('Mamedyarov, Shakhriyar', '2736', '2808', 0, 'w', '0.50', '-0.10')], '2015-10-01': [('Inants, Aghasi', '2396', '2796', 1, 'w', '1.00', '0.08'), ('Li, Ruifeng', '2446', '2796', 1, 'b', '0.50', '-0.39'), ('Priyadharshan, K.', '2462', '2796', 0, 'w', '0.50', '-0.38'), ('Sheng, Joshua', '2425', '2796', 0, 'b', '1.00', '0.10'), ('Azarov, Sergei', '2580', '2796', 0, 'w', '1.00', '0.22'), ('Durarbayli, Vasif', '2618', '2796', 0, 'b', '0.50', '-0.23'), ('Indjic, Aleksandar', '2569', '2796', 0, 'w', '1.00', '0.21'), ('Shimanov, Aleksandr', '2593', '2796', 0, 'b